In [ ]:
%pip install selenium
%pip install beautifulsoup4
%pip install pandas

In [ ]:
from selenium import webdriver
from datetime import timedelta, datetime
from bs4 import BeautifulSoup
import pandas as pd
from selenium.webdriver.common.by import By
import time
from selenium.webdriver.common.action_chains import ActionChains 
import re

# Парсинг данных

In [ ]:
driver = webdriver.Chrome()
driver.get('https://cryptobriefing.com/news/')

data = []

while True:
    html = driver.page_source
    soup = BeautifulSoup(html)

    containers = soup.find_all("li",attrs={"class":"main-news-item"})

    for i, d in enumerate(containers):
        #print(f"Analyzing containers. i = {i}")
        if i < len(data) - 1:
            continue
        d = d.find('section')
        d = d.find('div', {'class': "main-news-info"})

        header = d.find("h2", {"class": "main-news-title"})
        header = header.find("a", {"class": "main-news-link"})
        if header is None:
            header_text = "NOT FOUND"
            href = "NOT FOUND"
            #category = "NOT FOUND"
        else:
            header_text = header['aria-label'].strip()
            href = header['href'].strip()
            #category = header.text.strip()
        
        meta = d.find("p", {"class": "main-news-meta"})
        time_meta = meta.find("time")
        meta = meta.find("a", {"class": "main-news-category-link"})

        if meta is None:
            category = "NOT FOUND"
            dttime = "NOT FOUND"
        else:
            category = meta.text.strip()

        if time_meta is None:
            dttime = "NOT FOUND"
        else:
            dttime = time_meta['datetime']
            if dttime is None:
                dttime = time_meta['title']

        msg = d.find("p", {"class": "main-news-message"})
        if msg is None:
            brief = "NOT FOUND"
        else:
            brief = msg.text.strip()

        print(i, header_text, dttime, href, category, brief)
        data.append((header_text, dttime, href, category, brief)) 

        #data.append((header, timedata, ref))
        #print(f"Found header: {header}, date: {timedata}, ref: {ref}")

    button = driver.find_element(By.XPATH, '/html/body/main/div/div/div/section/div/a')
    location = button.location
    size = button.size
    w, h = size['width'], size['height']
    driver.execute_script(f"window.scrollTo(0, {h});")
    driver.execute_script("arguments[0].click();", button)
    time.sleep(1)


In [ ]:
df = pd.DataFrame(data, columns = ['header', 'date', 'link', 'category', 'brief'])
df = df.drop_duplicates()
df.tail()

In [ ]:
df.to_csv("../data/cryptobriefing_2025_2019_04_03.csv", index=False)